In [44]:
import pandas as pd

dt = pd.read_csv('unlimited_results.csv') 
df = pd.read_csv('concatenated_FFA.csv')
dt['distance'] = dt['distance'].fillna(0).astype(int)
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141204 entries, 0 to 141203
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   rank              141204 non-null  object 
 1   time              141204 non-null  object 
 2   athlete           141204 non-null  object 
 3   club              141204 non-null  object 
 4   category          141204 non-null  object 
 5   distance          141204 non-null  int64  
 6   date              141204 non-null  object 
 7   competition_type  141204 non-null  object 
 8   competition_name  141204 non-null  object 
 9   location          141204 non-null  object 
 10  ligue             141193 non-null  object 
 11  department        141204 non-null  float64
 12  type              71759 non-null   object 
 13  level             141204 non-null  object 
 14  Minute_Time       141204 non-null  float64
 15  vitesse           140948 non-null  float64
dtypes: float64(3), int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   results_url       293 non-null    object 
 1   competition_date  293 non-null    object 
 2   competition_type  293 non-null    object 
 3   competition_name  293 non-null    object 
 4   location          293 non-null    object 
 5   ligue             291 non-null    object 
 6   department        291 non-null    float64
 7   type              203 non-null    object 
 8   level             293 non-null    object 
dtypes: float64(1), object(8)
memory usage: 20.7+ KB


In [45]:
merged_df = pd.concat([dt, df], axis=0, ignore_index=True)

# Étape 2 : Supprimer les doublons basés sur 'competition_name' et 'distance'
merged_df = merged_df.drop_duplicates(subset=['competition_name', 'distance'])

# Étape 3 : Compter le nombre de coureurs par 'competition_name'
nombre_coureurs = dt.groupby('competition_name').size().reset_index(name='Nombre_Coureur')

# Étape 4 : Grouper par 'competition_name' et concaténer les distances uniques
aggregated_df = (
    merged_df
    .groupby('competition_name', as_index=False)
    .agg({
        'distance': lambda x: ', '.join(sorted(set(map(str, x.dropna())))),  # Exclure les NaN ici
        **{col: 'first' for col in merged_df.columns if col not in ['competition_name', 'distance']}
    })
)

# Étape 5 : Fusionner avec le nombre de coureurs et remplacer NaN par 0
aggregated_df = pd.merge(aggregated_df, nombre_coureurs, on='competition_name', how='left')
aggregated_df['Nombre_Coureur'] = aggregated_df['Nombre_Coureur'].fillna(0).astype(int)

# Étape 6 : Supprimer les colonnes non désirées
aggregated_df = aggregated_df.drop(columns=['club', 'athlete', 'time', 'rank', 'Minute_Time', 'vitesse',
                                            'results_url', 'competition_date', 'category'])

# Étape 7 : Sauvegarder le résultat dans un fichier CSV
aggregated_df.to_csv('resultat_course.csv', index=False)